In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR, SVC

from xgboost import XGBRegressor

/home/maebee/.pyenv/versions/lewagon/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df = pd.read_csv(os.path.join('data', 'player_pairs_stats.csv'))
df.drop_duplicates(subset=['match_id'], inplace=True)
df

,match_id,player,player_deaths_per_min,player_assists_per_min,player_tower_damage,player_xp_per_min,player_gold_per_min,player_net_worth,player_kills_per_min,player_hero_damage,...,opponent_xp_per_min,opponent_gold_per_min,opponent_net_worth,opponent_kills_per_min,opponent_hero_damage,opponent_last_hits,opponent_hero_id,opponent_roshans_killed,opponent_obs_placed,winner
0,6880924102,767434930,0.005068,0.008616,243,530,381,12019,0.003548,15961,...,558,453,13757,0.002027,12523,216,42,0.0,0.0,767434930
1,6880924214,153110997,0.009366,0.002755,861,394,269,6678,0.001653,10518,...,810,555,16532,0.005510,15334,227,44,0.0,0.0,1121170451
2,6880924314,118839691,0.002921,0.003895,102,361,369,6440,0.000000,3073,...,253,278,4919,0.000974,6766,19,119,0.0,0.0,437601765
3,6880924403,124272893,0.003254,0.002169,70,671,388,10066,0.002711,9104,...,496,326,8841,0.002711,11215,62,51,0.0,0.0,233422854
4,6880924409,147428574,0.005543,0.004989,262,486,463,11465,0.001109,22946,...,422,314,8097,0.000554,10193,31,87,0.0,0.0,292686154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7786,6882526312,209521221,0.005394,0.000674,0,296,240,5213,0.000674,3279,...,455,364,8389,0.004720,13284,57,25,0.0,0.0,275868212
7787,6882531210,149338363,0.003709,0.011869,68,371,243,4923,0.002226,5885,...,257,226,3905,0.000742,6862,50,36,0.0,0.0,149338363
7788,6882541503,436911654,0.001385,0.004155,295,343,535,6579,0.005540,6297,...,223,428,4758,0.000000,4072,12,9,0.0,0.0,436911654
7789,6882541917,461384275,0.001927,0.005780,0,176,214,2389,0.000000,514,...,433,496,4983,0.005780,1919,35,41,0.0,0.0,136365412


In [3]:
for row in range(len(df)):
    if df['player'][row] == df['winner'][row]:
        df['winner'][row] = 1
    else:
        df['winner'][row] = 0
df

/tmp/ipykernel_7339/1772519859.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['winner'][row] = 1
/tmp/ipykernel_7339/1772519859.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['winner'][row] = 0


,match_id,player,player_deaths_per_min,player_assists_per_min,player_tower_damage,player_xp_per_min,player_gold_per_min,player_net_worth,player_kills_per_min,player_hero_damage,...,opponent_xp_per_min,opponent_gold_per_min,opponent_net_worth,opponent_kills_per_min,opponent_hero_damage,opponent_last_hits,opponent_hero_id,opponent_roshans_killed,opponent_obs_placed,winner
0,6880924102,767434930,0.005068,0.008616,243,530,381,12019,0.003548,15961,...,558,453,13757,0.002027,12523,216,42,0.0,0.0,1
1,6880924214,153110997,0.009366,0.002755,861,394,269,6678,0.001653,10518,...,810,555,16532,0.005510,15334,227,44,0.0,0.0,0
2,6880924314,118839691,0.002921,0.003895,102,361,369,6440,0.000000,3073,...,253,278,4919,0.000974,6766,19,119,0.0,0.0,0
3,6880924403,124272893,0.003254,0.002169,70,671,388,10066,0.002711,9104,...,496,326,8841,0.002711,11215,62,51,0.0,0.0,0
4,6880924409,147428574,0.005543,0.004989,262,486,463,11465,0.001109,22946,...,422,314,8097,0.000554,10193,31,87,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7786,6882526312,209521221,0.005394,0.000674,0,296,240,5213,0.000674,3279,...,455,364,8389,0.004720,13284,57,25,0.0,0.0,0
7787,6882531210,149338363,0.003709,0.011869,68,371,243,4923,0.002226,5885,...,257,226,3905,0.000742,6862,50,36,0.0,0.0,1
7788,6882541503,436911654,0.001385,0.004155,295,343,535,6579,0.005540,6297,...,223,428,4758,0.000000,4072,12,9,0.0,0.0,1
7789,6882541917,461384275,0.001927,0.005780,0,176,214,2389,0.000000,514,...,433,496,4983,0.005780,1919,35,41,0.0,0.0,0


In [4]:
X = df.drop(columns=['match_id', 'player', 'player_hero_id', 'opponent', 'opponent_hero_id', 'winner'])
y = df['winner']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)
